### 3.2.3 Word2Vec (Word Embedding)

- Author: Phanxuan Phuc
- Project: https://github.com/phanxuanphucnd/TensorFlow-2.0-Tutorial

In [1]:
import os
import random
import zipfile
import numpy as np
import collections
import urllib.request
import tensorflow as tf

2021-07-06 10:35:07.623683: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-07-06 10:35:07.623714: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
# Training parameters
lr = 0.1
bs = 128
eval_step = 200000
display_step = 10000
num_steps = 3000000

# Evaluation parameters
eval_words = ['five', 'of', 'going', 'hardware', 'american', 'britain']

# Word2Vec parameters
embedding_size = 200     # Dimension of the embedding vector
max_vocab_size = 50000   # Total numbers of different words in the Vocaburary
min_occurrence = 10      # Remove all words that doesn't appears at least n times
skip_window = 3          # How many words to consider left and right
num_skips = 2            # How many times to reuse an input to generate a label
num_sampled = 64         # Number of negative examples to sample

### Download Dataset

- Text8 Dataset: [Download](http://mattmahoney.net/dc/text8.zip)

In [3]:
# Unzip the dataset file. Text has already been processed.

data_path = '../../data/text8.zip'

with zipfile.ZipFile(data_path) as f:
    text_words = f.read(f.namelist()[0]).lower().split()

In [4]:
# Build the Dictionary and replace rare words with UNK token.
count = [('UNK', -1)]

# Retrieve the most common words.
count.extend(collections.Counter(text_words).most_common(max_vocab_size - 1))

# Remove samples with less than 'min_occurence' ocurrences.
for i in range(len(count) - 1, -1, -1):
    if count[i][1] < min_occurrence:
        count.pop(i)
    else:
        # The collection is ordered, so that stop when 'min_occurence' is reached
        break
        
# Compute the Vocaburary size
vocab_size = len(count)

# Assign an id to each word
word2id = dict()
for i, (word, _) in enumerate(count):
    word2id[word] = i
    
data = list()
unk_count = 0
for word in text_words:
    # Retrieve a word id, or assign it index 0 ('UNK') if not in Dictionary
    index = word2id.get(word, 0)
    if index == 0:
        unk_count += 1
    data.append(index)
    
id2word = dict(zip(word2id.values(), word2id.keys()))


print(f"Words count: {len(text_words)}")
print(f"Unique words: {len(set(text_words))}")
print(f"Vocaburary size: {vocab_size}")

Words count: 17005207
Unique words: 253854
Vocaburary size: 47135


In [5]:
data_index = 0

# Generate training batch for the skip-gram model
def next_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size), dtype=np.int32)
    
    # Get window_size (words left and right + current one)
    span = 2 * skip_window + 1
    buffer = collections.deque(maxlen=span)
    if data_index + span > len(data):
        data_index = 0
    
    buffer.extend(data[data_index: data_index + span])
    data_index += span
    
    for i in range(batch_size // num_skips):
        context_words = [w for w in range(span) if w !=  skip_window]
        word_to_use = random.sample(context_words, num_skips)
        for j, context_word in enumerate(word_to_use):
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[context_word]
            
        if data_index == len(data):
            buffer.extend(data[0: span])
            data_index = span
        else:
            buffer.append(data[data_index])
            data_index += 1
    
    # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    
    return batch, labels